In [2]:
import math
import numpy as np
import pandas as pd
from ces_direct_energy_utils import *

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
## scenario switch
scenario = 'opt' # can be one of {opt, constrained, global}
## parameter values
theta = 4             # scopevec for comparative advantage
sigma = 1      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1  # elasticity of demand for each individual manufactured good j at Foreign
sigmaE = 1
sigmaEstar = 1

# reminants from older version, please do not change
logit = 0 # 1 means logit estimations of supply elasticity; 0 means fixed elasticities

# beta gamma not used in the code unless logit indicated
beta=1.892412
gamma=0.807998928

In [1]:
# carbon content per unit of energy
h1 = 1

# elasticity of supply of renewables
epsilonSstar1 = 0.5
epsilonS1 = 0.5

# baseline, no renewable energy
epsilonSvec = [(epsilonS1, h1, 1)]
epsilonSstarvec = [(epsilonSstar1, h1, 1)]
assert(sum(k for i,j,k in epsilonSvec) == 1)
assert(sum(k for i,j,k in epsilonSstarvec) == 1)

In [5]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)

# comment out if running simulation with no renewables
#df = pd.read_csv("../../raw_data/BaselineCarbon_2015.csv", index_col=['regionbase'],header='infer')
#df = pd.read_csv("../../raw_data/BaselineCarbon_direct_cons.csv", index_col=['regionbase'],header='infer')
df = pd.read_excel("../../raw_data/baselinecarbon_direct_2018.xlsx", index_col = 0)
df['jxbar']=df['Cex']/(df['Cex'] + df['Ceystar'])
df['jmbar']=df['Cey']/(df['Cey'] + df['Cem'])

In [6]:
if scenario == 'constrained':
    df = df[(df['region_scenario'] == 3)] 
    tax_scenario= pd.DataFrame({'tax_sce': ['purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid']})
elif scenario == 'global':
    df = df[(df['region_scenario'] == 3)] 
    tax_scenario= pd.DataFrame({'tax_sce': ['global']})
elif scenario == 'opt':
    df = df[df['region_scenario'] != 4]
    tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral']})
df

,region_scenario,Qe,Qestar,Qeworld,CeHH_old,Cem,Cex,CeFF_old,Ce,Cestar,...,Gestar,Ceworld,Geworld,Ced,Cedstar,Cey,Ceystar,World total Cec,jxbar,jmbar
regionbase,,,,,,,,,,,,,,,,,,,,,
US as Home,1,4.71140,28.92390,33.6353,4.5281,1.2126,0.4605,27.4341,5.7407,27.8946,...,28.6467,33.6353,33.6353,1.0741,3.6166,3.4540,23.8175,4.6907,0.018968,0.740153
EU28 as Home,2,0.86580,32.76950,33.6353,2.8788,1.1005,0.5741,29.0819,3.9793,29.6560,...,30.1824,33.6353,33.6353,0.8961,3.7946,1.9827,25.2873,4.6907,0.022199,0.643066
OECD37 as Home,3,9.30397,24.33133,33.6353,11.2795,2.4744,1.0261,18.8553,13.7539,19.8814,...,21.3297,33.6353,33.6353,2.5272,2.1635,8.7523,16.6918,4.6907,0.057913,0.779597
China as Home,5,7.54910,26.08620,33.6353,8.2294,0.7909,1.6864,22.9286,9.0203,24.6150,...,23.7195,33.6353,33.6353,0.5039,4.1868,7.7255,18.7418,4.6907,0.082553,0.907132
OECD and China as Home,6,16.85307,16.78223,33.6353,20.8579,1.9164,1.3636,9.4974,22.7743,10.8610,...,11.4138,33.6353,33.6353,3.0311,1.6596,17.8268,7.8378,4.6907,0.148195,0.902934
US and EU as Home,7,5.57720,28.05810,33.6353,7.6116,2.1084,0.8299,23.0854,9.7200,23.9153,...,25.1938,33.6353,33.6353,1.9702,2.7205,5.6414,20.3649,4.6907,0.039156,0.727941


In [7]:
tax_scenario

,tax_sce
0,Unilateral


In [8]:
#tax_scenario= pd.DataFrame({'tax_sce': ['global','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid']})
#tax_scenario= pd.DataFrame({'tax_sce': ['purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid']})
#tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral','PC_hybrid'], 'Base':[0,1]},index=[1,2])
#tax_scenario= pd.DataFrame({'tax_sce': ['Unilateral', 'puretp', 'EP_hybrid']})
#tax_scenario= pd.DataFrame({'tax_sce': ['purete']})

In [9]:
def iterate_tax(tax_scenario,df,theta, sigma, sigmastar, sigmaE, sigmaEstar, epsilonSvec, epsilonSstarvec, beta, gamma, logit):
    paralist = (theta, sigma, sigmastar, sigmaE, sigmaEstar, epsilonSvec, epsilonSstarvec, beta, gamma, logit)
    varphilist = np.arange(0,5,0.1)
    #varphilist = np.arange (0,1,0.2)
    #varphilist = [0,1]
    output=[]
    prevtb = []
    for varphi in varphilist:    
        tax_df=df.apply(iterate_region, axis=1, raw=False, args=(prevtb, varphi, tax_scenario, paralist))
        prevtb = tax_df[['region_scenario','tb', 'pe', 'te', 'prop']]
        output.append(tax_df)
        print(varphi)
        
    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    
    # if extraction tax too large home stops extracting
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid' or tax_scenario['tax_sce'] == 'Unilateral':
        output.te[output.Qe_prime==0]=output.pe+output.tb
    print(tax_scenario['tax_sce'])
    return output

In [10]:
def iterate_region(df, prevtb, varphi, tax_scenario, paralist):
    # initial guess if previous value isn't available
    pe = 1
    te = 0.5
    tb_mat = [0,0.5]
    
    # use previous vector of solution if available
    if (type(prevtb) != list):
        curr_region = prevtb[prevtb['region_scenario'] == df['region_scenario']]
        vals = curr_region.values
        tb_mat = [vals[0][1], vals[0][4]]
        pe = vals[0][2]
        te = vals[0][3]
    tax_temp = tax_eq(pe, te, tb_mat, df, tax_scenario, varphi, paralist)
    tax_temp.opt_tax()
    ret = tax_temp.retrieve()
    return ret

In [11]:
res = tax_scenario.apply(iterate_tax, axis=1, args=(df,theta, sigma, sigmastar, sigmaE, sigmaEstar, epsilonSvec, epsilonSstarvec, beta, gamma, logit))

0
1
Unilateral


In [12]:
output_list=[]
for i in range(0,len(tax_scenario)):
    output_list.append(res.loc[i])
result = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
result.reset_index(level=0, inplace=True)
result

,tax_sce,regionbase,varphi,pe,tb,prop,te,jxbar_prime,jmbar_prime,j0_prime,...,delta_UCed,delta_UCedstar,leak,leakstar,welfare,welfare_noexternality,Qe1_prime,Qe1star_prime,region_scenario,conv
0,Unilateral,US as Home,0.0,1.000000,0.000000,0.5000,0.0,0.018968,0.740153,0.018968,...,-0.000000,-0.000000,0.0,0.0,-0.000000e+00,0.000000e+00,4.711400,28.923900,1.0,1.0
6,Unilateral,US as Home,1.0,0.982030,0.666520,0.0000,1.0,0.020455,0.740153,0.013971,...,-0.536938,0.065581,0.0,0.0,3.540702e+00,-2.718766e+00,3.794213,27.893724,1.0,1.0
1,Unilateral,EU28 as Home,0.0,1.000000,0.000000,0.5000,0.0,0.022199,0.643066,0.022199,...,-0.000000,-0.000000,0.0,0.0,-0.000000e+00,0.000000e+00,0.865800,32.769500,2.0,1.0
7,Unilateral,EU28 as Home,1.0,0.982351,0.681389,0.0000,1.0,0.023999,0.643066,0.016280,...,-0.456176,0.067569,0.0,0.0,3.680509e+00,-2.677807e+00,0.705369,31.623000,2.0,1.0
2,Unilateral,OECD37 as Home,0.0,1.000000,0.000000,0.5000,0.0,0.057913,0.779597,0.057913,...,-0.000000,-0.000000,0.0,0.0,-1.898713e-14,-1.898713e-14,9.303970,24.331330,3.0,1.0
8,Unilateral,OECD37 as Home,1.0,0.943492,0.684949,0.0000,1.0,0.062787,0.779597,0.042427,...,-1.232322,0.125845,0.0,0.0,3.481988e+00,-2.664672e+00,7.375657,21.659196,3.0,1.0
3,Unilateral,China as Home,0.0,1.000000,0.000000,0.5000,0.0,0.082553,0.907132,0.082553,...,-0.000000,-0.000000,0.0,0.0,-0.000000e+00,0.000000e+00,7.549100,26.086200,5.0,1.0
9,Unilateral,China as Home,1.0,0.964375,0.671944,1.0000,1.0,0.088849,0.907132,0.061491,...,-0.248145,0.151876,0.0,0.0,3.330787e+00,-2.574428e+00,6.021886,24.260669,5.0,1.0
4,Unilateral,OECD and China as Home,0.0,1.000000,0.000000,0.5125,0.0,0.148195,0.902934,0.148195,...,-0.000000,-0.000000,0.0,0.0,-0.000000e+00,0.000000e+00,16.853070,16.782230,6.0,1.0
10,Unilateral,OECD and China as Home,1.0,0.880305,0.708559,0.0000,1.0,0.161489,0.902934,0.108793,...,-1.403459,0.211577,0.0,0.0,3.272076e+00,-2.576136e+00,12.932625,13.005179,6.0,1.0


In [13]:
result[['pe','tb', 'Ced_prime', 'Cedstar_prime', 'Cey_prime', 'Cex_prime', 'Cem_prime', 'Ceystar_prime', 'Vg_prime']]

,pe,tb,Ced_prime,Cedstar_prime,Cey_prime,Cex_prime,Cem_prime,Ceystar_prime,Vg_prime
0,1.000000,0.000000,1.074100,3.616600,3.454000,0.460500,1.212600,23.817500,31.110667
6,0.982030,0.666520,0.651542,3.682779,2.095174,0.306319,0.735555,24.216567,31.110667
1,1.000000,0.000000,0.896100,3.794600,1.982700,0.574100,1.100500,25.287300,20.554667
7,0.982351,0.681389,0.538606,3.862775,1.191713,0.379575,0.661462,25.694237,20.554667
2,1.000000,0.000000,2.527200,2.163500,8.752300,1.026100,2.474400,16.691800,74.844667
8,0.943492,0.684949,1.551913,2.293077,5.374648,0.695734,1.519490,17.599991,74.844667
3,1.000000,0.000000,0.503900,4.186800,7.725500,1.686400,0.790900,18.741800,56.776000
9,0.964375,0.671944,0.307947,4.341464,4.721268,1.127770,0.483341,19.300762,56.776000
4,1.000000,0.000000,3.031100,1.659600,17.826800,1.363600,1.916400,7.837800,131.621333
10,0.880305,0.708559,1.907715,1.885255,11.219837,0.954305,1.206144,8.764549,131.621333


In [14]:
#outcome_t = Outcomes[Outcomes['regionbase'] == 'US as Home']
result[['pe','tb','te','varphi', 'Qe_prime', 'Qeworld_prime','delta_Vg','delta_Vgstar','delta_Le','delta_Lestar','delta_UCed', 'delta_UCedstar','welfare','welfare_noexternality']]

,pe,tb,te,varphi,Qe_prime,Qeworld_prime,delta_Vg,delta_Vgstar,delta_Le,delta_Lestar,delta_UCed,delta_UCedstar,welfare,welfare_noexternality
0,1.000000,0.000000,0.0,0.0,4.711400,33.635300,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000e+00,0.000000e+00
6,0.982030,0.666520,1.0,1.0,3.794213,31.687938,-2.332815,0.234678,-0.750221,-1.020948,-0.536938,0.065581,3.540702e+00,-2.718766e+00
1,1.000000,0.000000,0.0,0.0,0.865800,33.635300,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000e+00,0.000000e+00
7,0.982351,0.681389,1.0,1.0,0.705369,32.328369,-1.569558,0.200392,-0.132539,-1.136412,-0.456176,0.067569,3.680509e+00,-2.677807e+00
2,1.000000,0.000000,0.0,0.0,9.303970,33.635300,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-1.898713e-14,-1.898713e-14
8,0.943492,0.684949,1.0,1.0,7.375657,29.034853,-5.474401,0.549121,-1.556267,-2.597367,-1.232322,0.125845,3.481988e+00,-2.664672e+00
3,1.000000,0.000000,0.0,0.0,7.549100,33.635300,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000e+00,0.000000e+00
9,0.964375,0.671944,1.0,1.0,6.021886,30.282555,-4.193893,-0.031722,-1.239087,-1.793211,-0.248145,0.151876,3.330787e+00,-2.574428e+00
4,1.000000,0.000000,0.0,0.0,16.853070,33.635300,-0.000000,-0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000e+00,0.000000e+00
10,0.880305,0.708559,1.0,1.0,12.932625,25.937805,-9.141488,0.479585,-3.079169,-3.555801,-1.403459,0.211577,3.272076e+00,-2.576136e+00


In [15]:
result[['pe','tb','Lg_prime', 'Lgstar_prime']]

,pe,tb,Lg_prime,Lgstar_prime
0,1.000000,0.000000,22.182167,141.837233
6,0.982030,0.666520,22.434237,141.632663
1,1.000000,0.000000,14.488533,149.530867
7,0.982351,0.681389,14.813878,149.267116
2,1.000000,0.000000,55.410933,108.608467
8,0.943492,0.684949,56.016484,108.119158
3,1.000000,0.000000,53.334100,110.685300
9,0.964375,0.671944,54.235052,109.956419
4,1.000000,0.000000,108.745600,55.273800
10,0.880305,0.708559,109.610681,54.580648


In [16]:
name = 'direct_consumption_' + scenario + '.csv'
if epsilonSstar1 == 2:
    name = 'direct_consumption_higheps_' + scenario + '.csv'
result.to_csv('../../output/' + name, header=True)
name

'direct_consumption_higheps_opt.csv'